# Notebook de présentation du calcul du trending sur 24h

**Ces exemples contiennent beaucoup de .show(), le rendant inefficace. Pour le vrai code, se référer au fichier *Req24h.py* **

In [5]:
import time
import pandas as pd
import os
from pyspark import SparkContext, SparkConf
from pyspark.sql import HiveContext
from pyspark.sql.window import Window
import pyspark.sql.functions as F
from pyspark.sql.types import FloatType

On a besoin du Hive Context pour utiliser *window*.

In [9]:
hc = HiveContext(sc)

In [10]:
wikipediadata = hc.read.format("org.apache.spark.sql.cassandra").\
               load(keyspace="projet", table="wikipediadata")

Numéro du jour (entre 0 et 89) :

In [11]:
day = 61

In [12]:
# On veut les 24 dernieres heures de day
sums = wikipediadata.where(wikipediadata.day == day)
# Sous-ensemble de test
sums = sums.where((sums.page == 'Cadillac_Brougham') | (sums.page == 'Roald_Dahl'))

# On cache pour plus tard
sums.cache()

DataFrame[projectcode: string, page: string, day: int, hour: int, views: bigint]

In [23]:
# Attention, ça prend du temps !!
sums.show(20)

+-----------+-----------------+---+----+-----+
|projectcode|             page|day|hour|views|
+-----------+-----------------+---+----+-----+
|         en|Cadillac_Brougham| 61|   0|   14|
|         en|Cadillac_Brougham| 61|   1|   11|
|         en|Cadillac_Brougham| 61|   2|   10|
|         en|Cadillac_Brougham| 61|   3|    8|
|         en|Cadillac_Brougham| 61|  14|    7|
|         en|Cadillac_Brougham| 61|  15|   10|
|         en|Cadillac_Brougham| 61|  16|    7|
|         en|Cadillac_Brougham| 61|  17|    7|
|         en|Cadillac_Brougham| 61|  18|    6|
|         en|Cadillac_Brougham| 61|  19|    8|
|         en|Cadillac_Brougham| 61|  20|   11|
|         en|Cadillac_Brougham| 61|  21|   14|
|         en|Cadillac_Brougham| 61|  22|   15|
|         en|Cadillac_Brougham| 61|  23|    8|
|         it|       Roald_Dahl| 61|   9|   10|
|         it|       Roald_Dahl| 61|  10|    8|
|         it|       Roald_Dahl| 61|  12|    9|
|         it|       Roald_Dahl| 61|  13|    6|
|         it|

In [14]:
# on définit une windows := heure précédente
window_spec = \
    Window \
        .partitionBy(sums.projectcode, sums.page) \
        .orderBy(sums.hour.asc()) \
        .rowsBetween(-1, -1)

In [15]:
# on calcule la différence entre views(h) - views(h-1)
diffs = sums.withColumn('diff', sums.views - F.sum(sums.views).over(window_spec))

In [24]:
diffs.show(20)

+-----------+----------+---+----+-----+----+
|projectcode|      page|day|hour|views|diff|
+-----------+----------+---+----+-----+----+
|         de|Roald_Dahl| 61|   9|   19|   0|
|         de|Roald_Dahl| 61|  10|   23|   4|
|         de|Roald_Dahl| 61|  11|   24|   1|
|         de|Roald_Dahl| 61|  12|    7| -17|
|         de|Roald_Dahl| 61|  13|   18|  11|
|         de|Roald_Dahl| 61|  14|   10|  -8|
|         de|Roald_Dahl| 61|  15|    9|  -1|
|         de|Roald_Dahl| 61|  16|   10|   1|
|         de|Roald_Dahl| 61|  17|   17|   7|
|         de|Roald_Dahl| 61|  18|   19|   2|
|         de|Roald_Dahl| 61|  19|   13|  -6|
|         de|Roald_Dahl| 61|  20|   15|   2|
|         de|Roald_Dahl| 61|  21|   12|  -3|
|         de|Roald_Dahl| 61|  22|    7|  -5|
|         de|Roald_Dahl| 61|  23|   12|   5|
|         sv|Roald_Dahl| 61|   8|    6|   0|
|         sv|Roald_Dahl| 61|   9|    7|   1|
|         sv|Roald_Dahl| 61|  10|    8|   1|
|         sv|Roald_Dahl| 61|  12|    6|  -2|
|         

In [25]:
# on calcule les coefs à appliquer à chaque jour
coefs = pd.DataFrame({'hour': range(24)})
coefs['coef'] = 1. / (24. - coefs.hour)

coefs = hc.createDataFrame(coefs)
diffs = diffs.join(coefs, 'hour')

In [26]:
diffs.show()

+-----------+-----------------+---+----+-----+----+--------------------+
|projectcode|             page|day|hour|views|diff|                coef|
+-----------+-----------------+---+----+-----+----+--------------------+
|         en|Cadillac_Brougham| 61|   0|   14|   0|0.041666666666666664|
|         es|       Roald_Dahl| 61|   0|   39|   0|0.041666666666666664|
|         en|       Roald_Dahl| 61|   0|  208|   0|0.041666666666666664|
|         fr|       Roald_Dahl| 61|   0|    7|   0|0.041666666666666664|
|         en|Cadillac_Brougham| 61|   1|   11|  -3|0.043478260869565216|
|         es|       Roald_Dahl| 61|   1|   23| -16|0.043478260869565216|
|         en|       Roald_Dahl| 61|   1|  174| -34|0.043478260869565216|
|         en|Cadillac_Brougham| 61|   2|   10|  -1|0.045454545454545456|
|         es|       Roald_Dahl| 61|   2|   17|  -6|0.045454545454545456|
|         en|       Roald_Dahl| 61|   2|  173|  -1|0.045454545454545456|
|         en|Cadillac_Brougham| 61|   3|    8|  -2|

In [31]:
# on calcul le score de chaque jour
diffs = diffs.withColumn('sub_score', diffs.diff * diffs.coef)

In [32]:
diffs.show()

+-----------+-----------------+---+----+-----+----+--------------------+--------------------+
|projectcode|             page|day|hour|views|diff|                coef|           sub_score|
+-----------+-----------------+---+----+-----+----+--------------------+--------------------+
|         en|Cadillac_Brougham| 61|   0|   14|   0|0.041666666666666664|                 0.0|
|         es|       Roald_Dahl| 61|   0|   39|   0|0.041666666666666664|                 0.0|
|         en|       Roald_Dahl| 61|   0|  208|   0|0.041666666666666664|                 0.0|
|         fr|       Roald_Dahl| 61|   0|    7|   0|0.041666666666666664|                 0.0|
|         en|Cadillac_Brougham| 61|   1|   11|  -3|0.043478260869565216|-0.13043478260869565|
|         es|       Roald_Dahl| 61|   1|   23| -16|0.043478260869565216| -0.6956521739130435|
|         en|       Roald_Dahl| 61|   1|  174| -34|0.043478260869565216| -1.4782608695652173|
|         en|Cadillac_Brougham| 61|   2|   10|  -1|0.0454545

In [33]:
totals = diffs.groupby('projectcode', 'page').sum('views', 'sub_score')
# on normalise par la racine de la somme des views 
totals = totals.withColumn('score',
        totals['SUM(sub_score)'] / F.sqrt(totals['SUM(views)'])) \
        .orderBy(F.desc('score')) \
        .withColumnRenamed('SUM(views)', 'total_views') \
        .limit(10)

In [36]:
totals.show()

+-----------+----------+-----------+-------------------+--------------------+
|projectcode|      page|total_views|     SUM(sub_score)|               score|
+-----------+----------+-----------+-------------------+--------------------+
|         es|Roald_Dahl|        504| 26.122951444690578|   1.163608740920994|
|         it|Roald_Dahl|        159|  2.430844155844156| 0.19277848124258912|
|         de|Roald_Dahl|        215| 1.2931929181929178|  0.0881950250050873|
|         sv|Roald_Dahl|         34|0.07142857142857145|0.012249898938750635|
|         sl|Roald_Dahl|          6|                0.0|                 0.0|
|         ca|Roald_Dahl|          6|                0.0|                 0.0|
|         cs|Roald_Dahl|          8|                0.0|                 0.0|
|         pt|Roald_Dahl|         19|               -1.0|-0.22941573387056174|
|         nl|Roald_Dahl|        253| -4.143012543012542| -0.2604689669297204|
|         no|Roald_Dahl|        378| -7.923171273171274| -0.4075

In [37]:
# Pour ces 10 premiers, on récupère la liste de leur vues
views = sums.select('projectcode', 'page', 'hour', 'views') \
        .join(totals.select('projectcode', 'page', 'total_views', 'score'), 
              (totals.projectcode == sums.projectcode) & (totals.page == sums.page), 'right_outer')

In [38]:
views.show()

+-----------+----------+----+-----+-----------+----------+-----------+--------------------+
|projectcode|      page|hour|views|projectcode|      page|total_views|               score|
+-----------+----------+----+-----+-----------+----------+-----------+--------------------+
|         de|Roald_Dahl|   9|   19|         de|Roald_Dahl|        215| 0.08819502500508732|
|         de|Roald_Dahl|  10|   23|         de|Roald_Dahl|        215| 0.08819502500508732|
|         de|Roald_Dahl|  11|   24|         de|Roald_Dahl|        215| 0.08819502500508732|
|         de|Roald_Dahl|  12|    7|         de|Roald_Dahl|        215| 0.08819502500508732|
|         de|Roald_Dahl|  13|   18|         de|Roald_Dahl|        215| 0.08819502500508732|
|         de|Roald_Dahl|  14|   10|         de|Roald_Dahl|        215| 0.08819502500508732|
|         de|Roald_Dahl|  15|    9|         de|Roald_Dahl|        215| 0.08819502500508732|
|         de|Roald_Dahl|  16|   10|         de|Roald_Dahl|        215| 0.0881950

In [39]:
# On transfere tout en panda
df = totals.select('projectcode', 'page', 'total_views', 'score').toPandas()

In [42]:
df.head()

,projectcode,page,total_views,score,0,1,2,3,4,5,...,14,15,16,17,18,19,20,21,22,23
0,es,Roald_Dahl,504,1.163609,39,23,17,27,10,10,...,8,17,21,31,35,33,44,30,31,57
1,it,Roald_Dahl,159,0.192778,0,0,0,0,0,0,...,9,18,9,20,19,17,10,8,16,0
2,de,Roald_Dahl,215,0.088195,0,0,0,0,0,0,...,10,9,10,17,19,13,15,12,7,12
3,sv,Roald_Dahl,34,0.012250,0,0,0,0,0,0,...,7,0,0,0,0,0,0,0,0,0
4,sl,Roald_Dahl,6,0.000000,0,0,0,0,0,0,...,0,0,0,0,0,6,0,0,0,0


In [50]:
df2 = views.toPandas()

In [51]:
df2.head(10)

,projectcode,page,hour,views,projectcode,page,total_views,score
0,de,Roald_Dahl,9,19,de,Roald_Dahl,215,0.088195
1,de,Roald_Dahl,10,23,de,Roald_Dahl,215,0.088195
2,de,Roald_Dahl,11,24,de,Roald_Dahl,215,0.088195
3,de,Roald_Dahl,12,7,de,Roald_Dahl,215,0.088195
4,de,Roald_Dahl,13,18,de,Roald_Dahl,215,0.088195
5,de,Roald_Dahl,14,10,de,Roald_Dahl,215,0.088195
6,de,Roald_Dahl,15,9,de,Roald_Dahl,215,0.088195
7,de,Roald_Dahl,16,10,de,Roald_Dahl,215,0.088195
8,de,Roald_Dahl,17,17,de,Roald_Dahl,215,0.088195
9,de,Roald_Dahl,18,19,de,Roald_Dahl,215,0.088195


In [52]:
# on réorganise
df2 = df2.iloc[:, 2:]
df2 = df2.pivot_table(values='views', columns=['hour'], index=['projectcode', 'page'], fill_value=0)

In [54]:
df2.head(10)

,hour,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
projectcode,page,,,,,,,,,,,,,,,,,,,,,
ca,Roald_Dahl,0,0,0,0,0,0,0,0,0,0,...,0,0,6,0,0,0,0,0,0,0
cs,Roald_Dahl,0,0,0,0,0,0,0,0,8,0,...,0,0,0,0,0,0,0,0,0,0
de,Roald_Dahl,0,0,0,0,0,0,0,0,0,19,...,10,9,10,17,19,13,15,12,7,12
es,Roald_Dahl,39,23,17,27,10,10,6,0,0,6,...,8,17,21,31,35,33,44,30,31,57
it,Roald_Dahl,0,0,0,0,0,0,0,0,0,10,...,9,18,9,20,19,17,10,8,16,0
nl,Roald_Dahl,0,0,0,0,0,0,0,0,19,9,...,23,16,20,26,20,24,22,14,10,0
no,Roald_Dahl,0,0,0,0,0,0,0,9,19,47,...,14,15,40,26,13,12,8,26,15,9
pt,Roald_Dahl,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,10,9
sl,Roald_Dahl,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,6,0,0,0,0


In [55]:
df = df.merge(df2, left_on=['projectcode', 'page'], right_index=True)
#df.to_csv('day_{}_24htrending.csv'.format(day), index=False)

In [56]:
df

,projectcode,page,total_views,score,0_x,1_x,2_x,3_x,4_x,5_x,...,14_y,15_y,16_y,17_y,18_y,19_y,20_y,21_y,22_y,23_y
0,es,Roald_Dahl,504,1.163609,39,23,17,27,10,10,...,8,17,21,31,35,33,44,30,31,57
1,it,Roald_Dahl,159,0.192778,0,0,0,0,0,0,...,9,18,9,20,19,17,10,8,16,0
2,de,Roald_Dahl,215,0.088195,0,0,0,0,0,0,...,10,9,10,17,19,13,15,12,7,12
3,sv,Roald_Dahl,34,0.012250,0,0,0,0,0,0,...,7,0,0,0,0,0,0,0,0,0
4,sl,Roald_Dahl,6,0.000000,0,0,0,0,0,0,...,0,0,0,0,0,6,0,0,0,0
5,ca,Roald_Dahl,6,0.000000,0,0,0,0,0,0,...,0,0,6,0,0,0,0,0,0,0
6,cs,Roald_Dahl,8,0.000000,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,pt,Roald_Dahl,19,-0.229416,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,10,9
8,nl,Roald_Dahl,253,-0.260469,0,0,0,0,0,0,...,23,16,20,26,20,24,22,14,10,0
9,no,Roald_Dahl,378,-0.407524,0,0,0,0,0,0,...,14,15,40,26,13,12,8,26,15,9


In [ ]:
# On efface le cache
hc.clearCache()